In [592]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

Cargar de CSV

In [593]:
df = pd.read_csv('Data/US_Crime_Data.csv')

Diccionario para reemplazar datos de la columna de State

In [594]:
key = ['ak','al','ar','az','ca','co','ct','de','fl','ga','hi','ia','id','il','in','ks',
'ky','la','ma','md','me','mi','mn','mo','ms','mt','nc','nd','ne','nh','nj','nm','nv','ny',
'oh','ok','or','pa','ri','sc','sd','tn','tx','ut','va','vt','wa','wi','wv','wy']

value = ['Alaska', 'Alabama', 'Arkansas', 'Arizona', 'California', 'Colorado', 'Connecticut',
         'Delaware', 'Florida', 'Georgia', 'Hawai', 'Iowa', 'Idaho', 'Illinois',
         'Indiana', 'Kansas', 'Kentucky', 'Louisiana', 'Massachusetts', 'Maryland', 'Maine', 
         'Michigan', 'Minnesota', 'Missouri', 'Misisipi', 'Montana', 'North Carolina', 'North Dakota',
         'Nebraska', 'New Hampshire', 'New Jersey', 'New México', 'Nevada', 'New York', 'Ohio','Oklahoma',
         'Oregon', 'Pensilvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas',
         'Utah', 'Virginia', 'Vermont', 'Washington', 'Wisconsin', 'West Virginia', 'Wyoming']

dic = dict(zip(key, value))

Correcion el la columna de State  "Algunos de los stados no son estados son ciudades o ubicaciones"

In [595]:

filtro1 = (df['State'].str.lower() == 'DC'.lower() ) | (df['State'].str.lower() =='D.C.'.lower() ) | (df['State'].str.lower() =='District Of Columbia'.lower())
df[filtro1] = 'wa'

filtro2 = (df['State'].str.lower() =='Los Angeles, California'.lower()) | (df['State'].str.lower() =='Venice Beach, Ca 90291'.lower()) | (df['State'].str.lower() =="New York And California".lower()) 
df[filtro2] = 'ca'


filtro3 = (df['State'].str.lower() =='Chicago'.lower())
df[filtro3] = 'il'

filtro3 = (df['State'].str.lower() =='io'.lower())
df[filtro3] = 'ia'

Transformacion de la data

In [596]:
df['City'] = df['City'].str.capitalize()
df['State'] = df['State'].str.lower()
df['State']= df['State'].apply(lambda x : str(x).replace('.', ''))
""" Alexandria Va es una Ciudad de Virginia """
fil = df['State'] == 'alexandria va'
""" Se actualiza todos los State de alexandria va a Virgina """
df[fil] = 'Virginia'

In [597]:
""" Actualizacion de la columna State """
for i,j in enumerate(df['State']):
    if j in dic.keys():
        for k,m in enumerate(dic):
            if j == m:
                df['State'][i] = dic[m]
df['State'] = df['State'].str.title()

Funcion para obtener DataFrame de datos agrupados con columna de conteo

In [598]:
def ContarColum(df_count):
    df_count = pd.DataFrame({'Count': df_count}).reset_index()
    return df_count

Cantidad de crimenes por ciudad

In [599]:
cantidad_city = ContarColum(df.groupby(by='City').size())
""" Grafico de barras """
#plt.bar(cantidad_city['City'], cantidad_city['Count'])
#plt.show()

' Grafico de barras '

Cantidad de crimenes por Estado

In [600]:
cantidad_state = ContarColum(df.groupby(by=['State']).size())
""" Grafico de barras """
#plt.bar(cantidad_state['State'], cantidad_state['Count'])
#plt.show()


' Grafico de barras '

Creacion de Dimensiones

1.1 Dim States

In [602]:
""" Dim States """
df_dim_state = df.groupby(by='State').mean(numeric_only=True).reset_index()
df_dim_state.rename(columns={'State':'Name_State'},inplace=True)



""" Data Frame de Codes State """
df_codes_state = pd.DataFrame({'Code':key, 'State':value})

""" Merge para crear el Dim_States """
df_dim_state = df_dim_state.merge(df_codes_state, how = 'left',left_on='Name_State', right_on='State',)

""" Eliminar columna State porque es inecesaria """
df_dim_state.pop('State')
df_dim_state['Code'] = df_dim_state['Code'].str.upper()


1.2 Dim City